In [2]:
import pandas as pd
import s3fs
from smart_open import open
import boto3
from io import StringIO # python3; python2: BytesIO 
from boto3.s3.transfer import TransferConfig
import metrics
import torch
from transformers import *
import numpy as np

In [3]:
all_features = ["Language", "tweet_timestamp"]
train_language = pd.read_csv(open("s3://recsys-challenge-2020/training.tsv"), encoding="utf-8",
                    names = all_features, usecols= [7, 8], sep="\x01"
                   )
# train_sorted = train.sort_values(by=['tweet_timestamp']).reset_index()
# n_head = 90
# n_tail = 10

# gt = list(val_set.reply_bool)
# pr = [0.5] * len(gt)
# prauc = metrics.compute_prauc(pr,gt)
# rce = metrics.compute_rce(pr,gt)

In [4]:
train_language.Language.value_counts()

D3164C7FBCF2565DDF915B1B3AEFB1DC    60722973
22C448FF81263D4BAF2A176145EE9EAD    23652265
06D61DCBBE938971E1EA0C38BD9B5446    13273364
ECED8A16BE2A5E8871FD55F4842F16B1    10280788
B9175601E87101A984A50F8A62A1C374     7884887
                                      ...   
D18801336202297E6484F634CAC6592E         614
4690215948DBF6872B8ED1C2BC87B17E         590
B2235C8B73239FDC5780DD132419833A         363
2E18F6F53E3CF073911AF0A93BBE5373         138
515E873C86EE1577E75FA2387B7FA59E          14
Name: Language, Length: 66, dtype: int64

In [6]:
train_language.Language.nunique()

66

In [5]:
train_language.head()

,Language,tweet_timestamp
0,22C448FF81263D4BAF2A176145EE9EAD,1581258144
1,B9175601E87101A984A50F8A62A1C374,1581018910
2,22C448FF81263D4BAF2A176145EE9EAD,1581244866
3,22C448FF81263D4BAF2A176145EE9EAD,1581250124
4,22C448FF81263D4BAF2A176145EE9EAD,1580958805


In [ ]:
train_set = train_sorted.head(int(len(train_sorted)*(n_head/100)))
train_set['reply_bool'] = train_set.reply_timestamp.fillna(0)
train_set.loc[train_set.reply_bool != 0.0, 'reply_bool'] = 1.0

In [ ]:
train_set.to_csv('s3://recsys-challenge-2020/train_set_reply.csv', index = False)

In [ ]:
val_set = train_sorted.tail(int(len(train_sorted)*(n_tail/100)))
val_set['reply_bool'] = val_set.reply_timestamp.fillna(0)
val_set.loc[val_set.reply_bool != 0.0, 'reply_bool'] = 1.0

In [ ]:
val_set.to_csv('s3://recsys-challenge-2020/val_set_reply.csv', index = False)

In [7]:
val_set.head()

,index,text_ tokens,tweet_id,tweet_timestamp,engaging_user_id,reply_timestamp,reply_bool
133267715,11908278,101\t17713\t9405\t119152\t119002\t118823\t9531...,F1529D503347DA75EFFD67CC1D9DA0F9,1581486678,A4B1828B571271073C68DA93D1465484,NaN,0.0
133267716,54640167,101\t64002\t9247\t23665\t9251\t17342\t23990\t9...,41D145799B12B26F5C580B46D7A0B88E,1581486678,F3A4C8EB81ADF3685E8F693037808A2C,NaN,0.0
133267717,102671906,101\t56898\t137\t44592\t11090\t13503\t10157\t1...,94062C22EE1C754B558A39E206B116E4,1581486678,8BD011AB59AC04491687749245009C60,NaN,0.0
133267718,54937609,101\t137\t15595\t21840\t14590\t75268\t57493\t1...,D434D803B122A2C7206E0495A955CDC4,1581486678,F7BFBA56274FCBBBB76BDD36D0D6495B,NaN,0.0
133267719,38246969,101\t56898\t137\t13069\t93870\t10284\t11274\t1...,952589011D88FED05B4444CBBD90A172,1581486678,10F6E035959B9AC156CED11E0FAED566,NaN,0.0


In [8]:
len(val_set)

14807523

In [10]:
len(train_set)

133267714

In [9]:
val_set[val_set["text_ tokens"].str.contains("1\.0")]

,index,text_ tokens,tweet_id,tweet_timestamp,engaging_user_id,reply_timestamp,reply_bool
